# Python-1, Лекция 9

Лектор: Хайбулин Даниэль

Подготовил материал: Лущ Иван

Итак, сегодня мы поговорим про итераторы и генераторы

### Iterator и iterable

Рассмотрим объекты, по которым возможно осуществлять итерацию с помощью цикла `for`:

In [ ]:
for i in range(3, 8):
    ...

for line in open("data.csv"):
    ...

for key in {"apple": 10, "banana": 20, "cherry": 30}:
    ...

for char in "Python!":
    ...


Данные объекты могут значительно различаться по своей природе, однако их объединяет возможность итерации. Каждый из них реализует специальный метод `__iter__`, возвращающий объект-итератор (**iterator**), что позволяет использовать их в конструкции цикла `for`.

In [ ]:
iterable = [1, 2, 3]
iterator = iterable.__iter__()
iterator

Объект-итератор (**iterator**) должен реализовывать как минимум один обязательный метод — `__next__`.

In [ ]:
iterator.__next__()

In [ ]:
iterator.__next__()

In [ ]:
iterator.__next__()

In [ ]:
iterator.__next__()

Каждый вызов данного метода возвращает следующий элемент последовательности, связанной с данным **итератором**. Когда элементы итерируемой последовательности заканчиваются, при последующем вызове метода `__next__` выкидывается исключение `StopIteration`. Давайте перейдем к определениям.

**Iterable** — это объект, у которого определён специальный метод `__iter__`, возвращающий итератор. К числу стандартных итерируемых объектов в `Python` относятся такие типы, как `list`, `dict`, `range` и другие.

Данное определение верно, однако не полностью отражает все возможности, для создания итерируемых объектов. К этому вопросу мы вернёмся подробнее далее.

**Iterator** — это объект, который реализует обязательный метод `__next__` и метод `__iter__`, который возвращает `self`. При каждом вызове метода `__next__` возвращается следующий элемент; если элементы закончились, выбрасывается исключение `StopIteration`.

![](iter.jpg)

Зачем нужны итераторы?

- Итераторы обеспечивают стандартизированный и удобный механизм последовательного доступа к элементам коллекций, без необходимости вручную управлять индексами или структурой данных. Это позволяет осуществлять перебор элементов с помощью конструкций высокого уровня, таких как цикл `for`, что делает код более лаконичным и читаемым.
- Использование итераторов позволяет работать с большими или потенциально бесконечными последовательностями без загрузки всех элементов в память одновременно. Таким образом, итераторы способствуют эффективному использованию ресурсов и позволяют обрабатывать данные "на лету", что особенно важно при работе с потоками данных, файлами и генераторами.


Ранее мы вызывали dunder-методы `__next__` и `__iter__` напрямую. Однако такие методы не предназначены для непосредственного использования в пользовательском коде. Для получения итератора и последовательного доступа к элементам рекомендуется использовать встроенные функции-обёртки `iter` и `next`:

In [ ]:
iterable: list[int] = [1, 2, 3]
iterator = iter(iterable)
iterator

In [ ]:
next(iterator)

In [ ]:
next(iterator)

In [ ]:
next(iterator)

In [ ]:
next(iterator)

Где можно еще встретить итераторы?


Например, функция `zip` возвращает итератор:

In [ ]:
a = [1, 2, 3]
b = [1, 2, 3]
c = zip(a, b)
print(next(c))
print(next(c))
print(next(c))

А также есть функция `enumerate` - она делает нумерацию элементов, что можно впоследствие использовать внутри for:

In [ ]:
k: list[int] = [4, 5, 6]
k_e = enumerate(k)
print(next(k_e))
print(next(k_e))
print(next(k_e))

Чтобы проиллюстрировать, как осуществляется проход по итерируемому объекту с помощью цикла `for`, рассмотрим реализацию аналогичной логики с использованием цикла `while`:

In [ ]:
def process_object(value: int): ...


# for value in range(3, 8):
#     process_object(value)

sequence = range(3, 8)
iterator = iter(sequence)
while True:
    try:
        num = next(iterator)
    except StopIteration:
        break
    else:
        process_object(num)

Рассмотрим практическую задачу реализации собственного итератора, на примере создания аналога `range`:

In [ ]:
import typing as tp


class Range:
    def __init__(self, start: int = 0, end: int = 10, step: int = 1) -> None:
        assert type(start) is int and type(end) is int and type(step) is int
        self.start = start
        self.end = end
        self.step = step

    def __iter__(self) -> tp.Self:
        return self

    def __next__(self) -> int:
        if self.start >= self.end:
            raise StopIteration()
        self.start += self.step
        return self.start - self.step


for i in Range(1, 3, 1):
    print(i)

Класс `Range` реализует оба протокола:
1. `Iterable`:

    В данном классе метод `__iter__` возвращает сам объект (`self`), благодаря чему экземпляр класса `Range` сам является итерируемым объектом.

2. `Iterator`:

    В классе Range реализован метод `__next__`, который возвращает очередное значение диапазона либо возбуждает исключение `StopIteration`, когда диапазон исчерпан.



Как проверить объект на итерируемость:

In [ ]:
import typing as tp


def is_iterable(obj: tp.Any) -> bool:
    try:
        iter(obj)
    except TypeError:
        return False
    else:
        return True


rng = Range()
if is_iterable(rng):
    print("rng is iterable")


num = 1
if not is_iterable(num):
    print("num is not iterable")

Рассмотрим вторую форму `iter`: `iter(callable, sentinel)`:
- в этой форме `iter` создаёт итератор, который каждый раз вызывает переданную функцию (без аргументов) и возвращает её результат.
- итерация продолжается, пока результат вызова функции **не равен значению `sentinel`**
- как только функция вернёт значение, равное `sentinel`, итерация останавливается (генерируется `StopIteration`).

Рассмотрим чтение файла по кускам (чанкам) по 3 символа:

In [ ]:
from functools import partial


with open("file.txt", "r") as f:
    # f.read(3) читает из потока по 3 символа.
    # partial позволяет сделать функцию без аргументов: read_3().
    read_3 = partial(f.read, 3)

    # iter(read_3, '') будет вызывать read_3(), пока результат не будет равен '' (пустая строка, возвращается, когда поток дочитан до конца).
    for chunk in iter(read_3, ""):
        print(chunk, end=" ")

Для чего нужна эта форма?
- Позволяет легко и красиво реализовывать циклы до наступления условия без явных `while` с `break`.
- Снимает необходимость вручную проверять условие остановки — `sentinel` делается встроенной частью итерации.

Существует альтернативный способ, позволяющий объекту стать итерируемым: если у объекта определён метод `__getitem__`, позволяющий получать элементы по индексам, начиная с нуля, такой объект также будет рассматриваться как итерируемый:

In [ ]:
class Sequence[T]:
    def __init__(self, *args: T) -> None:
        self.args = args

    def __getitem__(self, index: int) -> T:
        if index < 0 or index >= len(self.args):
            raise IndexError(index)
        return self.args[index]

In [ ]:
seq = Sequence(1, 2, 3, 4, 5)
seq[0], seq[2], seq[4]

In [ ]:
for i in seq:
    print(i, end=" ")

Когда Python встречает конструкцию `for i in seq:`, он сначала ищет у объекта метод `__iter__`. Если такого нет, но присутствует метод `__getitem__` (и он корректно обрабатывает индексы, начиная с `0` и повышая их), Python начинает запрашивать элементы по индексам: `seq[0]`, `seq[1]`, ...
Когда выбрасывается исключение `IndexError`, это служит сигналом об окончании итерации.

In [ ]:
it = iter(seq)

In [ ]:
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))

Вывод:
В Python объект считается итерируемым (`iterable`), если он реализует хотя бы один из двух протоколов:
- **Протокол итерируемости (Iterable)** — наличие метода `__iter__`, который возвращает итератор.
- **Протокол последовательности (Sequence protocol)** — наличие метода `__getitem__` с поддержкой индексации с нуля.


Рассмотрим, возможно ли использование оператора принадлежности `in` класса `Sequence`:

In [ ]:
seq = Sequence(2, 3, 5, 8, 13, 21)

In [ ]:
8 in seq

In [ ]:
1 in seq

In [ ]:
1 in seq, 8 in seq

Такие же проверки сделаем с классом `Range`:

In [ ]:
rng = Range(1, 4, 1)

In [ ]:
1 in rng

In [ ]:
2 in rng

In [ ]:
3 in rng

In [ ]:
3 in rng

In [ ]:
4 in rng

А давайте проверим на `range`:

In [ ]:
rng = range(1, 4, 1)

In [ ]:
1 in rng

In [ ]:
2 in rng

In [ ]:
3 in rng

In [ ]:
3 in rng

In [ ]:
4 in rng

Если у объекта не определён явно `__contains__`, Python пробует пройти по объекту с помощью итерации (обычно через `__iter__` или через `__getitem__`). В этом случае `in` просто перебирает все элементы, сравнивая их с искомым:

In [ ]:
def __contains__[T](self, value: T):
    for item in self:
        if item == value:
            return True
    return False

Для того, чтобы наш `Range` работал корректно, следует добавить реализацию `__contains__`:

In [ ]:
import typing as tp


class Range:
    def __init__(self, start: int = 0, end: int = 10, step: int = 1) -> None:
        assert type(start) is int and type(end) is int and type(step) is int
        self.start = start
        self.end = end
        self.step = step

    def __iter__(self) -> tp.Self:
        return self

    def __next__(self) -> int:
        if self.start >= self.end:
            raise StopIteration()
        self.start += self.step
        return self.start - self.step

    def __contains__(self, value: int) -> bool:
        if self.step > 0:
            if value < self.start or value >= self.end:
                return False
        else:
            if value > self.start or value <= self.end:
                return False

        return (value - self.start) % self.step == 0


for i in Range(1, 3, 1):
    print(i)


rng = Range(1, 4, 1)
print(1 in rng)
print(2 in rng)
print(3 in rng)
print(3 in rng)
print(4 in rng)

### Генераторы

In [ ]:
def magic_generator() -> tp.Iterator[int]:
    yield 42
    yield 42
    yield 42
    yield 42
    yield 42

In [ ]:
gen = magic_generator()
type(gen)

In [ ]:
for i in magic_generator():
    print(i)

In [ ]:
for i in iter(magic_generator()):
    print(i)

In [ ]:
g = magic_generator()
iter(g) is g

In [ ]:
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

**Генератор** — это особый вид итератора, позволяющий итерироваться по последовательности данных без хранения полной последовательности в памяти. Генераторы генерируют объекты налету, что и делает их хорошим выбором для больших объёмов данных.

Генератор делится на три типа:

- **Генераторная функция** - функция, которая содержит в своем теле ключевое слово **yield**. 
- **Генератор** - тип, возвращаемый генераторной функцией.
- **Генераторные выражения** - похожи на вложения списков, однако объявляются круглыми скобками.

Каждый вызов метода `__next__` (или функции `next`) для генератора приводит к выполнению функции до следующей инструкции `yield`, после чего управление возвращается вызывающей стороне. Когда выполнение функции завершается, выкидывается исключение `StopIteration`, сигнализируя об окончании последовательности.

Рассмотрим задачу вычисления суммы квадратов чётных чисел в переданной последовательности. Для начала реализуем её, используя базовые конструкции языка Python, такие как цикл `for` и условный оператор:

In [ ]:
def sum_even_squares(iterable: tp.Iterable[int]) -> int:
    total = 0
    for element in iterable:
        if element % 2 == 0:
            total += element**2
    return total


seq = [1, 2, 3, 4, 5, 6]
print(sum_even_squares(seq))

Рассмотрим более структурированный подход к решению задачи, разделив функциональность фильтрации и преобразования элементов на две отдельные функции. Такой подход способствует улучшению читаемости, повторного использования кода и повышению его модульности:

In [ ]:
def even(iterable: tp.Iterable[int]) -> list[int]:
    result = []
    for i in iterable:
        if i % 2 != 0:
            continue
        result.append(i)
    return result


def squares(iterable: tp.Iterable[int]) -> list[int]:
    result = []
    for i in iterable:
        result.append(i**2)
    return result


seq = [1, 2, 3, 4, 5, 6]
sum(squares(even(seq)))

В типовых задачах обработки больших объёмов данных финальный этап обычно заключается в агрегировании результатов с помощью некоторой функции сокращения (например, подсчёт суммы, среднего, максимального значения и т.д.). В приведённой выше реализации данная логика также реализуется через последовательное применение фильтрации и агрегации. Однако такой подход требует хранения практически всей промежуточной коллекции в памяти, что может привести к существенным издержкам при работе с очень крупными или потенциально бесконечными последовательностями и вызвать проблемы, связанные с превышением доступного объёма оперативной памяти.

In [ ]:
%%time
# хотелось оценить потребление оперативной памяти в момент исполнения, для этого можно воспользовать top/htop
import sys

seq = squares(even(range(100_000_000)))
print(sys.getsizeof(seq))
sum(seq)

In [ ]:
def even(iterable: tp.Iterable[int]) -> tp.Iterator[int]:
    for elem in iterable:
        if elem % 2 == 0:
            yield elem


def squares(iterable: tp.Iterable[int]) -> tp.Iterator[int]:
    for elem in iterable:
        yield elem**2


seq = [1, 2, 3, 4, 5, 6]
sum(squares(even(seq)))

In [ ]:
%%time

import sys

seq = squares(even(range(100_000_000)))
print(sys.getsizeof(seq))
sum(seq)

Давайте попробуем реализовать следующий генератор:

`repeat(iterable, times=None)` — генератор, который итерируется по указанному объекту столько раз, сколько задано в параметре `times`. Если параметр `times` не указан, итератор будет бесконечно итерироваться по объекту.

In [ ]:
def repeat(iterable: tp.Iterable[int], times: int | None = None):
    for _ in range(times) if times is not None else iter(int, 1):
        for it in iterable:
            yield it


print(list(repeat([3], times=3)))

В Python существует более подходящая конструкция для передачи элементов другого итерируемого объекта через генератор, чем использование цикла `for` с последовательными инструкциями `yield`. Для этой цели предназначена конструкция `yield from`, которая обеспечивает прямую и лаконичную передачу значений из вложенного итерируемого объекта в вызывающий генератор:

In [ ]:
def repeat(iterable, times=None):
    for _ in range(times) if times is not None else iter(int, 1):
        yield from iterable


print(list(repeat([3], times=3)))

На практике вы, вероятно, уже использовали генераторы, даже не осознавая этого. Далее подробно рассмотрим генераторные выражения:

In [ ]:
squares = (x**2 for x in range(5))
squares

In [ ]:
for square in squares:
    print(square, end=" ")

In [ ]:
max(x for x in range(100_000_000) if x % 11 == 0)

Отрефакторим ранее представленный код, используя генераторные выражения:

In [ ]:
def even(iterable: tp.Iterable[int]) -> tp.Iterator[int]:
    return (element for element in iterable if element % 2 == 0)


def squares(iterable: tp.Iterable[int]) -> tp.Iterator[int]:
    return (element**2 for element in iterable)


seq = [1, 2, 3, 4, 5, 6]
sum(squares(even(seq)))

Мы можем использовать генератор, для реализации итератора нашего объекта, давайте это сделаем для нашего класса `Range`:

In [ ]:
import typing as tp


class Range:
    def __init__(self, start: int = 0, end: int = 10, step: int = 1) -> None:
        assert type(start) is int and type(end) is int and type(step) is int
        self.start = start
        self.end = end
        self.step = step

    def __iter__(self) -> tp.Iterator[int]:
        while self.start < self.end:
            yield self.start
            self.start += self.step

    def __contains__(self, value: int) -> bool:
        if self.step > 0:
            if value < self.start or value >= self.end:
                return False
        else:
            if value > self.start or value <= self.end:
                return False

        return (value - self.start) % self.step == 0


for i in Range(1, 3, 1):
    print(i)


rng = Range(1, 4, 1)
print(1 in rng)
print(2 in rng)
print(3 in rng)
print(3 in rng)
print(4 in rng)

Следует отметить, что для возвращаемого значения метода `__iter__` в приведённом примере была использована аннотация `tp.Iterator[int]`, что является корректным указанием типа итератора. Однако в модуле typing предусмотрен специальный тип — `Generator[YieldType, SendType, ReturnType]`, предназначенный для более точного описания генераторов, возвращаемых соответствующими функциями. Можно было вместо `tp.Iterator[int]` написать `tp.Generator[int, None, None]`. Предлагаю рассмотреть полный функционал генераторов:

In [ ]:
def magic_generator() -> tp.Iterator[int]:
    yield 42

In [ ]:
def magic_generator() -> tp.Generator[int, None, None]:
    yield 42

Рассмотрим расширение взаимодействия с генераторами, позволяя им не только отдавать данные наружу, но и принимать значения снаружи в ходе работы:

In [ ]:
def magic_generator() -> tp.Generator[int, int, None]:
    print("Give me a number, please")
    num = yield
    print(f"Got number: {num}")
    yield num**2
    print("Finished")

In [ ]:
g = magic_generator()
next(g)

`send` — это специальный метод генераторов в Python, который позволяет "послать" значение в генератор на месте текущего выражения `yield`.

In [ ]:
g.send(42)

In [ ]:
g.send(42)

Метод `close` предназначен для явного завершения работы генератора. При вызове этого метода генератор выбрасывает внутри себя исключение `GeneratorExit`. Генератор может обработать это исключение в своём теле.

In [ ]:
def accumulator() -> tp.Generator[int, int, None]:
    total = 0
    while True:
        try:
            value = yield total
        except GeneratorExit:
            print(f"Generator is close, {total=}")
            raise

        if value is None:
            break
        total += value


gen = accumulator()
print(next(gen))

print(gen.send(5))
print(gen.send(10))
print(gen.send(-4))

gen.close()

После вызова метода `close` дальнейшие попытки получения значений из генератора приведут к выбрасыванию исключения `StopIteration`.

In [ ]:
next(gen)

Метод `throw` у генераторов позволяет кинуть исключение непосредственно в точке, на которой генератор в данный момент приостановлен оператором `yield`.

In [ ]:
def accumulator() -> tp.Generator[int, int, None]:
    total = 0
    while True:
        try:
            value = yield total
        except GeneratorExit:
            print(f"Generator is closed, {total=}")
            raise
        except ValueError:
            print("Обнаружен ValueError! Обнуляю total.")
            total = 0
            continue
        except Exception as exc:
            print(f"Поймано исключение: {exc}. Завершаю генератор.")
            break

        if value is None:
            break
        total += value


gen = accumulator()
print(next(gen))
print(gen.send(10))
print(gen.send(5))

print(gen.throw(ValueError("Сброс суммы")))
print(gen.send(3))

print(gen.throw(RuntimeError("критическая ошибка")))
